In [1]:
import pandas as pd

Читаем метаданные корпуса, подправленные вручную и дополненные автоматически (паттерны, кол-ва токенов итд)

In [2]:
df = pd.read_csv('./corpus-augmented-replicated.csv',sep='\t',index_col=0)

## Метаданные корпуса:

`filename` - строка, полное название текста 

#### Свойства текстов
`date` - строка, дата подписания/написания/обращения 

`authors` - массив, автор документа или выступающий/подписавшийся ("ответственный за документ")

`genres` – массив, все жанровые теги (манифест/выступление/фрагмент)

`names` - массив, заявленное название (иногда больше чем одно напр. ['резолюция всесоюзной конференции', 'рекорды посевов'])

`topics` - массив, темы документа, заявленные в названии или размеченные вручную где возможно. где невозможно выделить одну большую тему - копия столбца names (считаем что документ с самостоятельной маленькой темой из названия)

#### Бинарные категории текстов:
`multiple_authors` - заявлено ли групповое авторство напр. декрет ленина + сталина – да; резолюция съезда - да; послание путина федеральному собранию – нет (считаем что коллективное авторство не заявлено)

`leader_in_author` - заявлено ли что лидер в авторах документа/в подписавшихся/выступает ли. список лидеров ссср [отсюда](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%80%D1%83%D0%BA%D0%BE%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_%D0%A1%D0%A1%D0%A1%D0%A0) - включаем маленкова. TODO подумать считается ли ельцин из 1990 уже руководителем 

`is_written_text` - текст публикуется/доставляется в письменном виде

`addressed_to_people` - непосредственное обращение к широкой аудитории (ко всем гражданам/ко всем рабочим/к казакам). если указано обратное то False. напр. манифест -- [да](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D1%81%D0%BE%D1%87%D0%B0%D0%B9%D1%88%D0%B8%D0%B9_%D0%BC%D0%B0%D0%BD%D0%B8%D1%84%D0%B5%D1%81%D1%82), послание федеральному собранию - нет

`regular_occasion` -- съезды/конференции/совет/послание фс/инаугурационная речь

`has_major_topic` - заявлена/очевидная единственная тема (см. `topics`)

`is_fragment` -- является ли текст извлечением из оригинального документа/речи

`distorted` - очевидные проблемы в ocr и проч 

#### Аггрегирующие признаки:
`year` - год текста

`period` - исторический период, связанный с правлением лидера страны

`period-denser` - более частые исторические периоды (напр довоенный сталин и послевоенный сталин)

`full_pattern` -- складывается из всех бинарных признаков 

`pattern` -- паттерны в модели где мы принимаем что жанр для наших целей полностью определяется признаками: `multiple_authors`, `leader_in_author`, `is_written_text`, `addressed_to_people` (последнее неочевидно, TODO посмотреть на экспериментах, возможно получится упростить схему удалив addressed_to_people)

#### Остальное
`token_count` - количество токенов (spacy)

`filename_raw` - вспомонательное поле для обхода корпуса



In [3]:
df

,filename,date,authors,genres,names,topics,multiple_authors,leader_in_authors,is_written_text,addressed_to_people,...,has_major_topic,pattern,year,is_fragment,distorted,full_pattern,token_count,filename_raw,period,period-denser
0,1894.10.20_николай ii_манифест_о восшествии на...,1894.10.20,['николай ii'],['манифест'],['о восшествии на престол'],['о восшествии на престол'],False,True,True,True,...,False,111,1894,False,False,1110000,478,1894.10.20_николай-ii_манифест_о-восшествии-на...,николай ii,николай ii
1,1896.01.01_николай ii_манифест_коронование,1896.01.01,['николай ii'],['манифест'],['коронование'],['коронование'],False,True,True,True,...,False,111,1896,False,False,1110000,202,1896.01.01_николай-ii_манифест_коронование.txt,николай ii,николай ii
2,1899.02.03_николай ii_манифест_законы княжеств...,1899.02.03,['николай ii'],['манифест'],['законы княжества финляндского'],['законы княжества финляндского'],False,True,True,True,...,False,111,1899,False,False,1110000,455,1899.02.03_николай-ii_манифест_законы-княжеств...,николай ii,николай ii
3,1903.02.26_николай ii_манифест_усовершенствова...,1903.02.26,['николай ii'],['манифест'],['усовершенствование государственной власти'],['усовершенствование государственной власти'],False,True,True,True,...,False,111,1903,False,False,1110000,679,1903.02.26_николай-ii_манифест_усовершенствова...,николай ii,николай ii
4,1904.01.27_николай ii_манифест_об открытии вое...,1904.01.27,['николай ii'],['манифест'],['об открытии военных действий против японии'],['русско японская война 1904'],False,True,True,True,...,True,111,1904,False,False,1110100,399,1904.01.27_николай-ii_манифест_об-открытии-вое...,николай ii,николай ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2024.02.29_путин_выступление.послание_послание...,2024.02.29,['путин'],"['выступление', 'послание']",['послание федеральному собранию'],['послание федеральному собранию'],False,True,False,False,...,False,100,2024,False,False,1001000,15982,2024.02.29_путин_выступление.послание_послание...,путин 2012-2024,путин 2018-2024
358,2024.03.14_путин_выступление.обращение к гражд...,2024.03.14,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],['выборы 2024'],False,True,False,True,...,True,101,2024,False,False,1011100,512,2024.03.14_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024
359,2024.03.21_путин_выступление.обращение к гражд...,2024.03.21,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],"['выборы 2024', 'итоги выборов']",False,True,False,True,...,True,101,2024,False,False,1011100,384,2024.03.21_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024
360,2024.03.23_путин_выступление.обращение к гражд...,2024.03.23,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],['теракт в крокусе'],False,True,False,True,...,True,101,2024,False,False,1010100,667,2024.03.23_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024


#### # поправим типы данных в массивах и булевых полях

In [4]:
import ast


In [5]:
def list_in_string_to_list(list_wrapped_in_string):
    return ast.literal_eval(list_wrapped_in_string)

In [6]:
df['pattern'] = df['multiple_authors'].astype(int).astype(str) \
+df['leader_in_authors'].astype(int).astype(str)\
+df['is_written_text'].astype(int).astype(str)\
+df['addressed_to_people'].astype(int).astype(str)
# +df['regular_occasion'].astype(int).astype(str)\
# +df['has_major_topic'].astype(int).astype(str)\
# +df['is_fragment'].astype(int).astype(str)\
# +df['distorted'].astype(int).astype(str)

df['full_pattern'] = df['multiple_authors'].astype(int).astype(str) \
+df['leader_in_authors'].astype(int).astype(str)\
+df['is_written_text'].astype(int).astype(str)\
+df['addressed_to_people'].astype(int).astype(str)\
+df['regular_occasion'].astype(int).astype(str)\
+df['has_major_topic'].astype(int).astype(str)\
+df['is_fragment'].astype(int).astype(str)\
+df['distorted'].astype(int).astype(str)
df




,filename,date,authors,genres,names,topics,multiple_authors,leader_in_authors,is_written_text,addressed_to_people,...,has_major_topic,pattern,year,is_fragment,distorted,full_pattern,token_count,filename_raw,period,period-denser
0,1894.10.20_николай ii_манифест_о восшествии на...,1894.10.20,['николай ii'],['манифест'],['о восшествии на престол'],['о восшествии на престол'],False,True,True,True,...,False,0111,1894,False,False,01110000,478,1894.10.20_николай-ii_манифест_о-восшествии-на...,николай ii,николай ii
1,1896.01.01_николай ii_манифест_коронование,1896.01.01,['николай ii'],['манифест'],['коронование'],['коронование'],False,True,True,True,...,False,0111,1896,False,False,01110000,202,1896.01.01_николай-ii_манифест_коронование.txt,николай ii,николай ii
2,1899.02.03_николай ii_манифест_законы княжеств...,1899.02.03,['николай ii'],['манифест'],['законы княжества финляндского'],['законы княжества финляндского'],False,True,True,True,...,False,0111,1899,False,False,01110000,455,1899.02.03_николай-ii_манифест_законы-княжеств...,николай ii,николай ii
3,1903.02.26_николай ii_манифест_усовершенствова...,1903.02.26,['николай ii'],['манифест'],['усовершенствование государственной власти'],['усовершенствование государственной власти'],False,True,True,True,...,False,0111,1903,False,False,01110000,679,1903.02.26_николай-ii_манифест_усовершенствова...,николай ii,николай ii
4,1904.01.27_николай ii_манифест_об открытии вое...,1904.01.27,['николай ii'],['манифест'],['об открытии военных действий против японии'],['русско японская война 1904'],False,True,True,True,...,True,0111,1904,False,False,01110100,399,1904.01.27_николай-ii_манифест_об-открытии-вое...,николай ii,николай ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2024.02.29_путин_выступление.послание_послание...,2024.02.29,['путин'],"['выступление', 'послание']",['послание федеральному собранию'],['послание федеральному собранию'],False,True,False,False,...,False,0100,2024,False,False,01001000,15982,2024.02.29_путин_выступление.послание_послание...,путин 2012-2024,путин 2018-2024
358,2024.03.14_путин_выступление.обращение к гражд...,2024.03.14,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],['выборы 2024'],False,True,False,True,...,True,0101,2024,False,False,01011100,512,2024.03.14_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024
359,2024.03.21_путин_выступление.обращение к гражд...,2024.03.21,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],"['выборы 2024', 'итоги выборов']",False,True,False,True,...,True,0101,2024,False,False,01011100,384,2024.03.21_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024
360,2024.03.23_путин_выступление.обращение к гражд...,2024.03.23,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],['теракт в крокусе'],False,True,False,True,...,True,0101,2024,False,False,01010100,667,2024.03.23_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024


In [7]:
import re
from spacy.lang.ru import Russian
import spacy

In [16]:
!python -m spacy download ru_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.7.0/ru_core_news_sm-3.7.0-py3-none-any.whl (15.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [8]:
nlp = spacy.load('ru_core_news_sm')

In [9]:
nlp.max_length = 150000000

In [10]:
stopwords_spacy = nlp.Defaults.stop_words
stopwords_spacy

{'а',
 'авось',
 'ага',
 'агу',
 'аж',
 'ай',
 'али',
 'алло',
 'ау',
 'ах',
 'ая',
 'б',
 'бац',
 'без',
 'безусловно',
 'бишь',
 'благо',
 'благодаря',
 'ближайшие',
 'близко',
 'более',
 'больше',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будучи',
 'будь',
 'будьте',
 'бы',
 'бывает',
 'бывала',
 'бывали',
 'бываю',
 'бывают',
 'был',
 'была',
 'были',
 'было',
 'бытует',
 'быть',
 'в',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вдали',
 'вдобавок',
 'вдруг',
 'ведь',
 'везде',
 'вернее',
 'весь',
 'взаимно',
 'взаправду',
 'видно',
 'вишь',
 'включая',
 'вместо',
 'внакладе',
 'вначале',
 'вне',
 'вниз',
 'внизу',
 'вновь',
 'во',
 'вовсе',
 'возможно',
 'воистину',
 'вокруг',
 'вон',
 'вообще',
 'вопреки',
 'вот',
 'вперекор',
 'вплоть',
 'вполне',
 'вправду',
 'вправе',
 'впрочем',
 'впрямь',
 'вресноту',
 'вроде',
 'вряд',
 'все',
 'всегда',
 'всего',
 'всей',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всея',
 'всю',
 'всюд

In [11]:
import os.path
from pathlib import Path

In [13]:
doc = nlp('. – !')
for token in doc:
    print(token.tag_)

PUNCT
PUNCT
SPACE
PUNCT


In [22]:
# lemmatization, stopwords filter
def lemmatize_texts(filenames_list
                 ):

    for filename in filenames_list:
        real_filename = '../corpus/'+filename
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                doc = nlp(f.read().lower()) 
            lemmatized_text = [token.lemma_ for token in doc if token.lemma_ not in stopwords_spacy]
            
            lemmas_folder = f"../corpus-lemmatized-filtered/"
            Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        

            with open(lemmas_folder+filename, 'w+') as f:
                f.write((' ').join(lemmatized_text))
        print(real_filename)
        
    return real_filename

In [23]:
lemmatize_texts(df['filename_raw'].to_list())

../corpus/1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt
../corpus/1896.01.01_николай-ii_манифест_коронование.txt
../corpus/1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt
../corpus/1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt
../corpus/1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt
../corpus/1904.12.12_николай-ii_указ_о-предначертаниях-к-усовершенствованию-государственного-порядка.txt
../corpus/1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt
../corpus/1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt
../corpus/1906.07.06_николай-ii_манифест_роспуск-думы-1.txt
../corpus/1907.06.03_николай-ii_манифест_роспуск-думы-2.txt
../corpus/1914.07.20_николай-ii_манифест_об-объявлении-военных-действий-между-россией-и-германией_1-мировая-война.txt
../corpus/1914.08.01_николай-ii_воззвание_воззвание-к-полякам_1-мировая-война.txt

'../corpus/2024.05.07_путин_выступление_выступление-на-церемонии-вступления-в-должность-президента_инаугурация.txt'

In [14]:
df

,filename,date,authors,genres,names,topics,multiple_authors,leader_in_authors,is_written_text,addressed_to_people,...,has_major_topic,pattern,year,is_fragment,distorted,full_pattern,token_count,filename_raw,period,period-denser
0,1894.10.20_николай ii_манифест_о восшествии на...,1894.10.20,['николай ii'],['манифест'],['о восшествии на престол'],['о восшествии на престол'],False,True,True,True,...,False,0111,1894,False,False,01110000,478,1894.10.20_николай-ii_манифест_о-восшествии-на...,николай ii,николай ii
1,1896.01.01_николай ii_манифест_коронование,1896.01.01,['николай ii'],['манифест'],['коронование'],['коронование'],False,True,True,True,...,False,0111,1896,False,False,01110000,202,1896.01.01_николай-ii_манифест_коронование.txt,николай ii,николай ii
2,1899.02.03_николай ii_манифест_законы княжеств...,1899.02.03,['николай ii'],['манифест'],['законы княжества финляндского'],['законы княжества финляндского'],False,True,True,True,...,False,0111,1899,False,False,01110000,455,1899.02.03_николай-ii_манифест_законы-княжеств...,николай ii,николай ii
3,1903.02.26_николай ii_манифест_усовершенствова...,1903.02.26,['николай ii'],['манифест'],['усовершенствование государственной власти'],['усовершенствование государственной власти'],False,True,True,True,...,False,0111,1903,False,False,01110000,679,1903.02.26_николай-ii_манифест_усовершенствова...,николай ii,николай ii
4,1904.01.27_николай ii_манифест_об открытии вое...,1904.01.27,['николай ii'],['манифест'],['об открытии военных действий против японии'],['русско японская война 1904'],False,True,True,True,...,True,0111,1904,False,False,01110100,399,1904.01.27_николай-ii_манифест_об-открытии-вое...,николай ii,николай ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2024.02.29_путин_выступление.послание_послание...,2024.02.29,['путин'],"['выступление', 'послание']",['послание федеральному собранию'],['послание федеральному собранию'],False,True,False,False,...,False,0100,2024,False,False,01001000,15982,2024.02.29_путин_выступление.послание_послание...,путин 2012-2024,путин 2018-2024
358,2024.03.14_путин_выступление.обращение к гражд...,2024.03.14,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],['выборы 2024'],False,True,False,True,...,True,0101,2024,False,False,01011100,512,2024.03.14_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024
359,2024.03.21_путин_выступление.обращение к гражд...,2024.03.21,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],"['выборы 2024', 'итоги выборов']",False,True,False,True,...,True,0101,2024,False,False,01011100,384,2024.03.21_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024
360,2024.03.23_путин_выступление.обращение к гражд...,2024.03.23,['путин'],"['выступление', 'обращение к гражданам', 'тв']",['обращение к гражданам'],['теракт в крокусе'],False,True,False,True,...,True,0101,2024,False,False,01010100,667,2024.03.23_путин_выступление.обращение-к-гражд...,путин 2012-2024,путин 2018-2024


In [51]:
df[
    (df['date'] < '1915')
]['filename_raw'].to_list()

['1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt',
 '1896.01.01_николай-ii_манифест_коронование.txt',
 '1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt',
 '1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt',
 '1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt',
 '1904.12.12_николай-ii_указ_о-предначертаниях-к-усовершенствованию-государственного-порядка.txt',
 '1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt',
 '1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt',
 '1906.07.06_николай-ii_манифест_роспуск-думы-1.txt',
 '1907.06.03_николай-ii_манифест_роспуск-думы-2.txt',
 '1914.07.20_николай-ii_манифест_об-объявлении-военных-действий-между-россией-и-германией_1-мировая-война.txt',
 '1914.08.01_николай-ii_воззвание_воззвание-к-полякам_1-мировая-война.txt',
 '1914.10.20_николай-ii_манифест_об-объявлении-войны-турции_1-мировая-в

In [56]:
# lemmatization, stopwords filter
def copy_and_rename_subcorpus_texts(filenames_list, filename_prefix
                 ):

    for filename in filenames_list:
        real_filename = '../corpus-lemmatized-filtered/'+filename
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                doc = f.read()
            lemmas_folder = f"../corpus-lemmatized-filtered-20y-window/"
            lemmas_folder_nested = f"../corpus-lemmatized-filtered-20y-window-nested/{filename_prefix}/"
            Path(lemmas_folder).mkdir(parents=True, exist_ok=True)  
            Path(lemmas_folder_nested).mkdir(parents=True, exist_ok=True)        

            with open(lemmas_folder+filename_prefix+'_'+filename, 'w+') as f:
                f.write(doc)
            with open(lemmas_folder_nested+filename, 'w+') as f:
                f.write(doc)
        print(real_filename)
        
    return real_filename

In [15]:
filenames_1894_1914 = df[
    (df['date'] <= '1914')
]['filename_raw'].to_list()
filenames_1894_1914


['1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt',
 '1896.01.01_николай-ii_манифест_коронование.txt',
 '1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt',
 '1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt',
 '1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt',
 '1904.12.12_николай-ii_указ_о-предначертаниях-к-усовершенствованию-государственного-порядка.txt',
 '1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt',
 '1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt',
 '1906.07.06_николай-ii_манифест_роспуск-думы-1.txt',
 '1907.06.03_николай-ii_манифест_роспуск-думы-2.txt']

In [60]:
copy_and_rename_subcorpus_texts(filenames_list=filenames_1894_1914, filename_prefix='1894-1914')

../corpus-lemmatized-filtered/1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt
../corpus-lemmatized-filtered/1896.01.01_николай-ii_манифест_коронование.txt
../corpus-lemmatized-filtered/1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt
../corpus-lemmatized-filtered/1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt
../corpus-lemmatized-filtered/1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt
../corpus-lemmatized-filtered/1904.12.12_николай-ii_указ_о-предначертаниях-к-усовершенствованию-государственного-порядка.txt
../corpus-lemmatized-filtered/1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt
../corpus-lemmatized-filtered/1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt
../corpus-lemmatized-filtered/1906.07.06_николай-ii_манифест_роспуск-думы-1.txt
../corpus-lemmatized-filtered/1907.06.03_николай-ii_манифест_роспуск-думы-2.txt

'../corpus-lemmatized-filtered/1907.06.03_николай-ii_манифест_роспуск-думы-2.txt'

In [61]:
for i in range(1914, 2034, 20):
    filenames_to_copy = df[
        (df['date'] > str(i))
        &
        (df['date'] <= str(i+20))
    ]['filename_raw'].to_list()
    copy_and_rename_subcorpus_texts(filenames_list=filenames_to_copy, filename_prefix=f'{i+1}-{i+20}')
    

../corpus-lemmatized-filtered/1914.07.20_николай-ii_манифест_об-объявлении-военных-действий-между-россией-и-германией_1-мировая-война.txt
../corpus-lemmatized-filtered/1914.08.01_николай-ii_воззвание_воззвание-к-полякам_1-мировая-война.txt
../corpus-lemmatized-filtered/1914.10.20_николай-ii_манифест_об-объявлении-войны-турции_1-мировая-война.txt
../corpus-lemmatized-filtered/1917.03.02_николай-ii_манифест_отречение.txt
../corpus-lemmatized-filtered/1917.03.03_временное-правительство_декларация.обращение-к-гражданам_декларация-о-составе-и-задачах.txt
../corpus-lemmatized-filtered/1917.03.03_временное-правительство_телеграмма.за-границу_о-взятии-власти.txt
../corpus-lemmatized-filtered/1917.03.06_временное-правительство_обращение-к-гражданам.письменное_создание-временного-правительства.txt
../corpus-lemmatized-filtered/1917.03.22_временное-правительство_постановление.власть_об-отмене-вероисповедных-и-национальных-ограничений.txt
../corpus-lemmatized-filtered/1917.04.12_временное-правител

In [16]:
from os import listdir
from os.path import isfile, join


In [18]:
onlyfiles = [f for f in listdir('../corpus-lemmatized-filtered-20y-window-nested/1894-1914') if isfile(join('../corpus-lemmatized-filtered-20y-window-nested/1894-1914', f))]
onlyfiles

['1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt',
 '1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt',
 '1907.06.03_николай-ii_манифест_роспуск-думы-2.txt',
 '1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt',
 '1906.07.06_николай-ii_манифест_роспуск-думы-1.txt',
 '1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt',
 '1904.12.12_николай-ii_указ_о-предначертаниях-к-усовершенствованию-государственного-порядка.txt',
 '1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt',
 '1896.01.01_николай-ii_манифест_коронование.txt',
 '1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt']

In [22]:
def count_lemmas_for_one_file(filename):
    lemmas_list = []
    if Path(filename).is_file():
        with open(filename, 'r') as f:
            doc = nlp(f.read().lower()) 
        for token in doc:
            if token.lemma_ not in stopwords_spacy:
                if token.tag_ not in ['SPACE','PUNKT']:
                    lemmas_list.append(token.lemma_)
        print(filename)
        
    return lemmas_list

In [23]:
basefolder = '../corpus-lemmatized-filtered-20y-window-nested'
lemmas_basefolder = '../lemmatized-20y-window'
subfolders = listdir(basefolder) 
for subfolder in subfolders:
    print(subfolder)
    files = listdir((join(basefolder, subfolder)))

    lemmas_for_period = []
    

    for filename in files:
        print(filename)
        lemmas_for_text = count_lemmas_for_one_file(join(basefolder, subfolder, filename))
        lemmas_for_period += lemmas_for_text 
    
    Path(join(lemmas_basefolder, subfolder)).mkdir(parents=True, exist_ok=True)   
    with open(join(lemmas_basefolder, subfolder, 'lemmas'), 'w+') as f:
        f.write((' ').join(lemmas_for_period))
        print('done', join(lemmas_basefolder, subfolder))



    

1955-1974
1966.04.08_съезд-кпсс-23.другие_выступление_обсуждение-отчетного-доклада.txt
../corpus-lemmatized-filtered-20y-window-nested/1955-1974/1966.04.08_съезд-кпсс-23.другие_выступление_обсуждение-отчетного-доклада.txt
1961.10.31_съезд-кпсс-22_резолюция.фрагмент_по-отчету-цк.txt
../corpus-lemmatized-filtered-20y-window-nested/1955-1974/1961.10.31_съезд-кпсс-22_резолюция.фрагмент_по-отчету-цк.txt
1959.02.05_съезд-кпсс-21_резолюция.фрагмент_по-докладу-контрольные-цифры-народного-хозяйства.txt
../corpus-lemmatized-filtered-20y-window-nested/1955-1974/1959.02.05_съезд-кпсс-21_резолюция.фрагмент_по-докладу-контрольные-цифры-народного-хозяйства.txt
1966.04.08_брежнев_выступление_заключительное-слово_съезд-кпсс-23.txt
../corpus-lemmatized-filtered-20y-window-nested/1955-1974/1966.04.08_брежнев_выступление_заключительное-слово_съезд-кпсс-23.txt
1961.10.17_хрущев_выступление.отчет_отчетный-доклад-цк_съезд-кпсс-22.txt
../corpus-lemmatized-filtered-20y-window-nested/1955-1974/1961.10.17_хрущев

In [ ]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        lemmas = count_lemmas(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        

        with open(lemmas_folder+'/noun.txt', 'w+') as f:
            f.write(('\n').join(lemmas['noun']))
        with open(lemmas_folder+'/adj.txt', 'w+') as f:
            f.write(('\n').join(lemmas['adj']))
        with open(lemmas_folder+'/verb.txt', 'w+') as f:
            f.write(('\n').join(lemmas['verb']))

        

In [ ]:
def count_lemmas(filenames_list
                 ):
    lemmas_dict={
                  'noun': [],
                  'verb': [],
                  'adj': []  
    }
    for filename in filenames_list:
        real_filename = '../corpus/'+re.sub(r' ', '-', filename, flags=re.IGNORECASE)+'.txt'
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                doc = nlp(f.read().lower()) 
                for token in doc:
                    if token.lemma_ not in stopwords_spacy:
                        if token.tag_ == 'NOUN':
                            lemmas_dict['noun'].append(token.lemma_)
                        elif token.tag_ == 'ADJ':
                            lemmas_dict['adj'].append(token.lemma_)
                        elif token.tag_ == 'VERB':
                            lemmas_dict['verb'].append(token.lemma_)
        print(real_filename)
        
    return lemmas_dict

In [15]:
def count_lemmas(filenames_list
                 ):
    lemmas_dict={
                  'noun': [],
                  'verb': [],
                  'adj': []  
    }
    for filename in filenames_list:
        real_filename = '../corpus/'+re.sub(r' ', '-', filename, flags=re.IGNORECASE)+'.txt'
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                doc = nlp(f.read().lower()) 
                for token in doc:
                    if token.lemma_ not in stopwords_spacy:
                        if token.tag_ == 'NOUN':
                            lemmas_dict['noun'].append(token.lemma_)
                        elif token.tag_ == 'ADJ':
                            lemmas_dict['adj'].append(token.lemma_)
                        elif token.tag_ == 'VERB':
                            lemmas_dict['verb'].append(token.lemma_)
        print(real_filename)
        
    return lemmas_dict

In [16]:
def count_lemmas_total(filenames_list
                 ):
    lemmas_list = []
    for filename in filenames_list:
        real_filename = '../corpus/'+re.sub(r' ', '-', filename, flags=re.IGNORECASE)+'.txt'
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                doc = nlp(f.read().lower()) 
                for token in doc:
                    if token.lemma_ not in stopwords_spacy:
                        if token.tag_ in ['NOUN','ADJ','VERB']:
                            lemmas_list.append(token.lemma_)
        print(real_filename)
        
    return lemmas_list

In [17]:
from razdel import sentenize

In [37]:
def text_to_lemmatized_sentences(filenames_list):
    
    lemmatized_sentences = []
    for filename in filenames_list:
        real_filename = '../corpus/'+re.sub(r' ', '-', filename, flags=re.IGNORECASE)+'.txt'
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                file_text = f.read()
            sentences = [re.sub(r'\n', '. ', _.text, flags=re.IGNORECASE) for _ in list(sentenize(file_text))] 

            lemmatized_sentences = [(' ').join([x.lemma_ for x in nlp(s.lower()) if x.lemma_ not in stopwords_spacy]) for s in sentences]

            
            
            
            # lemmatized_sentences = [x for s in sentences for x in nlp(s.lower()) if x.lemma_ not in stopwords_spacy ]
        print(real_filename)
        
    return lemmatized_sentences

In [40]:
def text_to_sentences(filenames_list):
    
    lemmatized_sentences = []
    for filename in filenames_list:
        real_filename = '../corpus/'+re.sub(r' ', '-', filename, flags=re.IGNORECASE)+'.txt'
        if Path(real_filename).is_file():
            with open(real_filename, 'r') as f:
                file_text = f.read()
            sentences = [re.sub(r'\n', '. ', _.text, flags=re.IGNORECASE) for _ in list(sentenize(file_text))] 
            
            # lemmatized_sentences = [x for s in sentences for x in nlp(s.lower()) if x.lemma_ not in stopwords_spacy ]
        print(real_filename)
        
    return sentences

In [41]:
test_sentences = text_to_sentences(['../corpus/2001.04.03_путин_выступление.послание_послание-федеральному-собранию'])
test_sentences

../corpus/../corpus/2001.04.03_путин_выступление.послание_послание-федеральному-собранию.txt


['Послание Президента Российской Федерации от 03.04.2001 г. б/н. (О положении в стране и основных направлениях внутренней и внешней политики государства). pravo.gov.ru.  . .  . . РОССИЙСКАЯ ФЕДЕРАЦИЯ.  . . ПОСЛАНИЕПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ ФЕДЕРАЛЬНОМУ СОБРАНИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.  . . (О положении в стране и основных направлениях внутренней и внешней политики государства).  . . ВЫСТУПЛЕНИЕ ПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ В.В.ПУТИНА С ПОСЛАНИЕМ ФЕДЕРАЛЬНОМУ СОБРАНИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.  . . Уважаемые депутаты Государственной Думы!',
 'Уважаемые члены Совета Федерации!',
 'Уважаемые граждане России!',
 'Представляя Послание на 2001 год, прежде всего несколько слов о годе минувшем.',
 'Стратегической задачей прошлого года было укрепление государства - государства в лице всех институтов и всех уровней власти.',
 'Было очевидно: без решения этой ключевой проблемы нам не достичь успехов ни в экономике, ни в социальной сфере.',
 'Мы поставили цель: выстроить четко работающую исполн

In [38]:
test_text_lemmatized = text_to_lemmatized_sentences(['../corpus/2001.04.03_путин_выступление.послание_послание-федеральному-собранию'])
test_text_lemmatized

../corpus/../corpus/2001.04.03_путин_выступление.послание_послание-федеральному-собранию.txt


['послание президент российский федерация 03.04.2001 г. / . ( положение страна основный направление внутренний внешний политика государство ) . pravo.gov.ru .   . .   . . российский федерация .   . . посланиепрезидента российский федерация федеральный собрание российский федерация .   . . ( положение страна основный направление внутренний внешний политика государство ) .   . . выступление президент российский федерация в.в.путина послание федеральный собрание российский федерация .   . . уважаемые депутат государственный дума !',
 'уважаемые член совет федерация !',
 'уважаемые гражданин россия !',
 'представлять послание 2001 год , слово год минувший .',
 'стратегический задача прошлый год укрепление государство - государство лицо институт уровень власть .',
 'очевидный : решение ключевой проблема достигнуть успех ни экономика , ни социальный сфера .',
 'поставить цель : выстроить четко работать исполнительный вертикаль , добиться правовой дисциплина действенный судебный система .',
 

In [32]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
tokens = tokenizer.split('всё очень плохо')  # [('всё', None), ('очень', None), ('плохо', None)]

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [39]:
messages = test_text_lemmatized

results = model.predict(messages, k=2)

for message, sentiment in zip(messages, results):
    # привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
    # люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
    # малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}]
    print(message, '->', sentiment)

послание президент российский федерация 03.04.2001 г. / . ( положение страна основный направление внутренний внешний политика государство ) . pravo.gov.ru .   . .   . . российский федерация .   . . посланиепрезидента российский федерация федеральный собрание российский федерация .   . . ( положение страна основный направление внутренний внешний политика государство ) .   . . выступление президент российский федерация в.в.путина послание федеральный собрание российский федерация .   . . уважаемые депутат государственный дума ! -> {'neutral': 0.6442351341247559, 'negative': 0.3140605390071869}
уважаемые член совет федерация ! -> {'neutral': 0.771853506565094, 'negative': 0.14805719256401062}
уважаемые гражданин россия ! -> {'positive': 0.6442351341247559, 'neutral': 0.15611489117145538}
представлять послание 2001 год , слово год минувший . -> {'neutral': 0.5698626637458801, 'negative': 0.287777841091156}
стратегический задача прошлый год укрепление государство - государство лицо институт

In [42]:
messages = test_sentences

results = model.predict(messages, k=2)

for message, sentiment in zip(messages, results):
    # привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
    # люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
    # малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}]
    print(message, '->', sentiment)

Послание Президента Российской Федерации от 03.04.2001 г. б/н. (О положении в стране и основных направлениях внутренней и внешней политики государства). pravo.gov.ru.  . .  . . РОССИЙСКАЯ ФЕДЕРАЦИЯ.  . . ПОСЛАНИЕПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ ФЕДЕРАЛЬНОМУ СОБРАНИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.  . . (О положении в стране и основных направлениях внутренней и внешней политики государства).  . . ВЫСТУПЛЕНИЕ ПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ В.В.ПУТИНА С ПОСЛАНИЕМ ФЕДЕРАЛЬНОМУ СОБРАНИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.  . . Уважаемые депутаты Государственной Думы! -> {'neutral': 0.7606606483459473, 'negative': 0.2509227991104126}
Уважаемые члены Совета Федерации! -> {'neutral': 0.960371196269989, 'negative': 0.05034062638878822}
Уважаемые граждане России! -> {'positive': 0.4532718360424042, 'neutral': 0.20690405368804932}
Представляя Послание на 2001 год, прежде всего несколько слов о годе минувшем. -> {'neutral': 0.6654205918312073, 'negative': 0.38492217659950256}
Стратегической задачей прошлого года было укр

In [23]:
test_pattern_period = df[
    df['pattern'].isin(['0100'])
][
    df['period-denser'].isin(['горбачев'])
]['filename'].to_list()
test_pattern_period

/var/folders/bs/37p3rgss433755h614xs_rym0000gn/T/ipykernel_15683/2895774375.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_pattern_period = df[


['1986.02.25_горбачев_выступление.отчет.доклад_политический доклад_съезд кпсс 27',
 '1986.03.06_горбачев_выступление_заключительная речь_съезд кпсс 27',
 '1988.06.28_горбачев_выступление.доклад_задачи по углублению перестройки_партконференция19',
 '1988.07.01_горбачев_выступление_заключительная речь_партконференция19',
 '1988.12.07_горбачев_выступление_выступление в оон',
 '1989.04.27_ельцин_выступление_к 1 съезду народных депутатов_съезд народных депутатов 1',
 '1989.07.06_горбачев_выступление_выступление в совете европы',
 '1990.07.02_горбачев_выступление.доклад_политический отчет_съезд кпсс 28',
 '1990.11.13_ельцин_выступление.стенограмма.ocr_на 2 сессии верховного совета рсфср_сессия верховного совета рсфср',
 '1991.05.21_ельцин_выступление_на 4 съезде народных депутатов_съезд народных депутатов 4']

In [22]:
lemmas = count_lemmas(['1986.02.25_горбачев_выступление.отчет.доклад_политический доклад_съезд кпсс 27'])

../corpus/1986.02.25_горбачев_выступление.отчет.доклад_политический-доклад_съезд-кпсс-27.txt


In [ ]:


real_filename = '../corpus/'+re.sub(r' ', '-', filename, flags=re.IGNORECASE)+'.txt'

In [31]:
from pathlib import Path
lemmas_folder = "../lemmatized/горбачев/0100"
Path(lemmas_folder).mkdir(parents=True, exist_ok=True)


In [35]:
with open(lemmas_folder+'/noun.txt', 'w+') as f:
    f.write(('\n').join(lemmas['noun']))
with open(lemmas_folder+'/adj.txt', 'w+') as f:
    f.write(('\n').join(lemmas['adj']))
with open(lemmas_folder+'/verb.txt', 'w+') as f:
    f.write(('\n').join(lemmas['verb']))

In [39]:
periods = list(df['period-denser'].unique())

In [40]:
patterns = list(df[
    df['period-denser'] == periods[0]
]['pattern'].unique())
patterns

['0111', '0110']

In [25]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        lemmas = count_lemmas(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        

        with open(lemmas_folder+'/noun.txt', 'w+') as f:
            f.write(('\n').join(lemmas['noun']))
        with open(lemmas_folder+'/adj.txt', 'w+') as f:
            f.write(('\n').join(lemmas['adj']))
        with open(lemmas_folder+'/verb.txt', 'w+') as f:
            f.write(('\n').join(lemmas['verb']))

        


николай ii


0111

../corpus/1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt
../corpus/1896.01.01_николай-ii_манифест_коронование.txt
../corpus/1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt
../corpus/1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt
../corpus/1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt
../corpus/1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt
../corpus/1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt
../corpus/1906.07.06_николай-ii_манифест_роспуск-думы-1.txt
../corpus/1907.06.03_николай-ii_манифест_роспуск-думы-2.txt
../corpus/1914.07.20_николай-ii_манифест_об-объявлении-военных-действий-между-россией-и-германией_1-мировая-война.txt
../corpus/1914.08.01_николай-ii_воззвание_воззвание-к-полякам_1-мировая-война.txt
../corpus/1914.10.20_николай-ii_манифест_об-объявлении-войны-турции_1-мировая-война.

In [28]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        

        with open(lemmas_folder+'/total.txt', 'w+') as f:
            f.write(('\n').join(lemmas))

        


николай ii


0111

../corpus/1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt
../corpus/1896.01.01_николай-ii_манифест_коронование.txt
../corpus/1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt
../corpus/1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt
../corpus/1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt
../corpus/1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt
../corpus/1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt
../corpus/1906.07.06_николай-ii_манифест_роспуск-думы-1.txt
../corpus/1907.06.03_николай-ii_манифест_роспуск-думы-2.txt
../corpus/1914.07.20_николай-ii_манифест_об-объявлении-военных-действий-между-россией-и-германией_1-мировая-война.txt
../corpus/1914.08.01_николай-ii_воззвание_воззвание-к-полякам_1-мировая-война.txt
../corpus/1914.10.20_николай-ii_манифест_об-объявлении-войны-турции_1-мировая-война.

In [43]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        lemmas = text_to_lemmatized_sentences(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        

        with open(lemmas_folder+'/sentences.txt', 'w+') as f:
            f.write(('\n').join(lemmas))

        


николай ii


0111

../corpus/1894.10.20_николай-ii_манифест_о-восшествии-на-престол.txt
../corpus/1896.01.01_николай-ii_манифест_коронование.txt
../corpus/1899.02.03_николай-ii_манифест_законы-княжества-финляндского.txt
../corpus/1903.02.26_николай-ii_манифест_усовершенствование-государственной-власти.txt
../corpus/1904.01.27_николай-ii_манифест_об-открытии-военных-действий-против-японии_русско-японская-война-1904.txt
../corpus/1905.08.06_николай-ii_манифест_учреждение-госдумы-1.txt
../corpus/1905.10.17_николай-ii_манифест_об-усовершенствовании-государственного-порядка_революция-1905.txt
../corpus/1906.07.06_николай-ii_манифест_роспуск-думы-1.txt
../corpus/1907.06.03_николай-ii_манифест_роспуск-думы-2.txt
../corpus/1914.07.20_николай-ii_манифест_об-объявлении-военных-действий-между-россией-и-германией_1-мировая-война.txt
../corpus/1914.08.01_николай-ii_воззвание_воззвание-к-полякам_1-мировая-война.txt
../corpus/1914.10.20_николай-ii_манифест_об-объявлении-войны-турции_1-мировая-война.

In [30]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        # lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        # Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        
        lemmas = []
        with open(lemmas_folder+'/total.txt', 'r') as f:
            lemmas = (f.read()).split('\n')

        lemmas_cleaned = [morph.parse(lemma)[0].normal_form for lemma in lemmas]
        with open(lemmas_folder+'/total_cleaned.txt', 'r') as f:
            f.write(('\n').join(lemmas))

In [36]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        # lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        # Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        
        lemmas = []
        with open(lemmas_folder+'/total.txt', 'r') as f:
            lemmas = (f.read()).split('\n')

        _2grams = [
            f"{lemmas[i]} {lemmas[i+1]}" for i in range(len(lemmas)-1)
        ]
        _3grams = [
            f"{lemmas[i]} {lemmas[i+1]} {lemmas[i+2]}" for i in range(len(lemmas)-2)
        ]
        with open(lemmas_folder+'/total_2grams.txt', 'w+') as f:
            f.write(('\n').join(_2grams))
        with open(lemmas_folder+'/total_3grams.txt', 'w+') as f:
            f.write(('\n').join(_3grams))


николай ii


0111


0110


временное правительство


1011


1010


ленин - революция


1011


1010


1111


0110


1110


0111


0100


1000


ленин - поздние годы


1110


0100


1000


1001


1010


0111


0000


сталин - довоенные годы


1010


0000


0100


0111


1000


1011


сталин - великая отечественная война


0101


0110


0100


сталин - послевоенные годы


0110


0101


0100


1010


0010


маленков


0010


1010


хрущев


1010


0100


1000


брежнев


0100


1000


1010


1110


горбачев


0100


0000


1000


1010


0101


1110


ельцин 1991-1996


0101


1110


1010


0100


0110


ельцин 1996-1999


1010


0100


0101


путин 2000-2004


0100


0111


путин 2000-2008


0100


0111


медведев


0101


0100


0111


путин 2012-2018


0100


0101


0110


путин 2018-2024


0100


0101


0111


0010



In [10]:
from collections import Counter

In [43]:
with open('../lemmatized/горбачев/0100/total.txt', 'r') as f:
    lemmas = (f.read()).split('\n')
lemmas_length = len(lemmas)
most_common_lemmas = pd.DataFrame(
    [
        (
            lemma_count[0],
            lemma_count[1],
            lemma_count[1]/lemmas_length*1_000_000
            )
        for lemma_count in Counter(lemmas).most_common()
        ],columns=['lemma', 'count', 'relative_count']
    )
most_common_lemmas

,lemma,count,relative_count
0,-,500,9242.485859
1,партия,416,7689.748235
2,политический,364,6728.529705
3,новый,347,6414.285186
4,вопрос,343,6340.345299
...,...,...,...
7405,роrовая,1,18.484972
7406,понят,1,18.484972
7407,остановка,1,18.484972
7408,каз,1,18.484972


In [48]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        # lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        # Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        
        lemmas = []
        with open(lemmas_folder+'/total.txt', 'r') as f:
            lemmas = (f.read()).split('\n')

        lemmas_length = len(lemmas)
        most_common_lemmas = pd.DataFrame(
            [
                (
                    lemma_count[0],
                    lemma_count[1],
                    lemma_count[1]/lemmas_length*1_000_000
                    )
                for lemma_count in Counter(lemmas).most_common()
                ],columns=['lemma', 'count', 'relative_count']
            )
        most_common_lemmas.to_csv(lemmas_folder+'/freq_lemmas.csv')


николай ii


0111


0110


временное правительство


1011


1010


ленин - революция


1011


1010


1111


0110


1110


0111


0100


1000


ленин - поздние годы


1110


0100


1000


1001


1010


0111


0000


сталин - довоенные годы


1010


0000


0100


0111


1000


1011


сталин - великая отечественная война


0101


0110


0100


сталин - послевоенные годы


0110


0101


0100


1010


0010


маленков


0010


1010


хрущев


1010


0100


1000


брежнев


0100


1000


1010


1110


горбачев


0100


0000


1000


1010


0101


1110


ельцин 1991-1996


0101


1110


1010


0100


0110


ельцин 1996-1999


1010


0100


0101


путин 2000-2004


0100


0111


путин 2000-2008


0100


0111


медведев


0101


0100


0111


путин 2012-2018


0100


0101


0110


путин 2018-2024


0100


0101


0111


0010



In [30]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    most_common_lemmas_for_period = pd.DataFrame()
    
    lemmas_length_by_period = 0
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        # lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        # Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        
        lemmas = []
        with open(lemmas_folder+'/total.txt', 'r') as f:
            lemmas = (f.read()).split('\n')

        lemmas_length = len(lemmas)
        lemmas_length_by_period += lemmas_length
        most_common_lemmas = pd.DataFrame(
            [
                (
                    lemma_count[0],
                    lemma_count[1],
                    lemma_count[1]/lemmas_length*1_000_000
                    )
                for lemma_count in Counter(lemmas).most_common()
                ],columns=['lemma', 'count', 'relative_count']
            )
        if most_common_lemmas_for_period.empty:
            most_common_lemmas_for_period = most_common_lemmas.rename(columns={'count':f'count_{pattern}', 'relative_count': f'relative_count_{pattern}'})
        else:
            most_common_lemmas_for_period = pd.merge(left=most_common_lemmas_for_period, right=most_common_lemmas.rename(columns={'count':f'count_{pattern}', 'relative_count': f'relative_count_{pattern}'}), on='lemma', how='outer')

        most_common_lemmas.to_csv(lemmas_folder+'/freq_lemmas.csv')
        
    
    # most_common_lemmas_for_period["relative_count_by_period"] = most_common_lemmas_for_period["count_by_period"]/lemmas_length_by_period
    most_common_lemmas_for_period.to_csv(f"../lemmatized/{period}/freq_lemmas.csv")
    


николай ii


0111


0110


временное правительство


1011


1010


ленин - революция


1011


1010


1111


0110


1110


0111


0100


1000


ленин - поздние годы


1110


0100


1000


1001


1010


0111


0000


сталин - довоенные годы


1010


0000


0100


0111


1000


1011


сталин - великая отечественная война


0101


0110


0100


сталин - послевоенные годы


0110


0101


0100


1010


0010


маленков


0010


1010


хрущев


1010


0100


1000


брежнев


0100


1000


1010


1110


горбачев


0100


0000


1000


1010


0101


1110


ельцин 1991-1996


0101


1110


1010


0100


0110


ельцин 1996-1999


1010


0100


0101


путин 2000-2004


0100


0111


путин 2000-2008


0100


0111


медведев


0101


0100


0111


путин 2012-2018


0100


0101


0110


путин 2018-2024


0100


0101


0111


0010



In [48]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    
    df_period = pd.read_csv(f'../lemmatized/{period}/freq_lemmas.csv',index_col=0)

    count_df = df_period.iloc[:,[i for i in range(len(df_period.columns)) if df_period.columns[i].startswith('count_')]].fillna(0)
    df_period['count_by_period'] = count_df.apply(sum,axis=1)
    df_period = df_period.sort_values(by=['count_by_period'], ascending=False).reset_index()
    df_period.to_csv(f'../lemmatized/{period}/freq_lemmas.csv')

    
    


николай ii


временное правительство


ленин - революция


ленин - поздние годы


сталин - довоенные годы


сталин - великая отечественная война


сталин - послевоенные годы


маленков


хрущев


брежнев


горбачев


ельцин 1991-1996


ельцин 1996-1999


путин 2000-2004


путин 2000-2008


медведев


путин 2012-2018


путин 2018-2024



In [57]:
periods = list(df['period-denser'].unique())
most_common_lemmas_total = pd.DataFrame()
lemmas_length_total = 0

for period in periods:
    print(f'\n{period}\n')
    
    most_common_lemmas = pd.read_csv(f'../lemmatized/{period}/freq_lemmas.csv',index_col=0)[['lemma', 'count_by_period']]

    lemmas_length_by_period = most_common_lemmas['count_by_period'].sum()
    lemmas_length_total += lemmas_length_by_period


    if most_common_lemmas_total.empty:
            most_common_lemmas_total = most_common_lemmas.rename(columns={'count_by_period':f'count_by_period_{period}'})
    else:
        most_common_lemmas_total = pd.merge(left=most_common_lemmas_total, right=most_common_lemmas.rename(columns={'count_by_period':f'count_by_period_{period}'}), on='lemma', how='outer')


    # count_df = df_period.iloc[:,[i for i in range(len(df_period.columns)) if df_period.columns[i].startswith('count_')]].fillna(0)
    # df_period['count_by_period'] = count_df.apply(sum,axis=1)
    # df_period = df_period.sort_values(by=['count_by_period'], ascending=False).reset_index()
    # df_period.to_csv(f'../lemmatized/{period}/freq_lemmas.csv')
    
    # most_common_lemmas_for_period["relative_count_by_period"] = most_common_lemmas_for_period["count_by_period"]/lemmas_length_by_period
most_common_lemmas_total.to_csv(f"../lemmatized/freq_lemmas.csv")
    


николай ii


временное правительство


ленин - революция


ленин - поздние годы


сталин - довоенные годы


сталин - великая отечественная война


сталин - послевоенные годы


маленков


хрущев


брежнев


горбачев


ельцин 1991-1996


ельцин 1996-1999


путин 2000-2004


путин 2000-2008


медведев


путин 2012-2018


путин 2018-2024



In [69]:
df_total = pd.read_csv(f"../lemmatized/freq_lemmas.csv", index_col=0)
count_df = df_total.iloc[:,[i for i in range(len(df_total.columns)) if df_total.columns[i].startswith('count_')]].fillna(0)
df_total['count_total'] = count_df.apply(sum,axis=1)
df_total = df_total.sort_values(by=['count_total'], ascending=False).reset_index().drop(columns=['index'])
df_total.to_csv(f"../lemmatized/freq_lemmas.csv")


now all the same but with 2 and 3-grams

In [64]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    most_common_lemmas_for_period = pd.DataFrame()
    
    lemmas_length_by_period = 0
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        # lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        # Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        
        lemmas = []
        with open(lemmas_folder+'/total_2grams.txt', 'r') as f:
            lemmas = (f.read()).split('\n')

        lemmas_length = len(lemmas)
        lemmas_length_by_period += lemmas_length
        most_common_lemmas = pd.DataFrame(
            [
                (
                    lemma_count[0],
                    lemma_count[1],
                    lemma_count[1]/lemmas_length*1_000_000
                    )
                for lemma_count in Counter(lemmas).most_common()
                ],columns=['lemma', 'count', 'relative_count']
            )
        if most_common_lemmas_for_period.empty:
            most_common_lemmas_for_period = most_common_lemmas.rename(columns={'count':f'count_{pattern}', 'relative_count': f'relative_count_{pattern}'})
        else:
            most_common_lemmas_for_period = pd.merge(left=most_common_lemmas_for_period, right=most_common_lemmas.rename(columns={'count':f'count_{pattern}', 'relative_count': f'relative_count_{pattern}'}), on='lemma', how='outer')

        most_common_lemmas.to_csv(lemmas_folder+'/freq_2grams.csv')
        
    
    # most_common_lemmas_for_period["relative_count_by_period"] = most_common_lemmas_for_period["count_by_period"]/lemmas_length_by_period
    most_common_lemmas_for_period.to_csv(f"../lemmatized/{period}/freq_2grams.csv")
    


николай ii


0111


0110


временное правительство


1011


1010


ленин - революция


1011


1010


1111


0110


1110


0111


0100


1000


ленин - поздние годы


1110


0100


1000


1001


1010


0111


0000


сталин - довоенные годы


1010


0000


0100


0111


1000


1011


сталин - великая отечественная война


0101


0110


0100


сталин - послевоенные годы


0110


0101


0100


1010


0010


маленков


0010


1010


хрущев


1010


0100


1000


брежнев


0100


1000


1010


1110


горбачев


0100


0000


1000


1010


0101


1110


ельцин 1991-1996


0101


1110


1010


0100


0110


ельцин 1996-1999


1010


0100


0101


путин 2000-2004


0100


0111


путин 2000-2008


0100


0111


медведев


0101


0100


0111


путин 2012-2018


0100


0101


0110


путин 2018-2024


0100


0101


0111


0010



In [65]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    
    df_period = pd.read_csv(f'../lemmatized/{period}/freq_2grams.csv',index_col=0)

    count_df = df_period.iloc[:,[i for i in range(len(df_period.columns)) if df_period.columns[i].startswith('count_')]].fillna(0)
    df_period['count_by_period'] = count_df.apply(sum,axis=1)
    df_period = df_period.sort_values(by=['count_by_period'], ascending=False).reset_index()
    df_period.to_csv(f'../lemmatized/{period}/freq_2grams.csv')

    
    


николай ii


временное правительство


ленин - революция


ленин - поздние годы


сталин - довоенные годы


сталин - великая отечественная война


сталин - послевоенные годы


маленков


хрущев


брежнев


горбачев


ельцин 1991-1996


ельцин 1996-1999


путин 2000-2004


путин 2000-2008


медведев


путин 2012-2018


путин 2018-2024



In [66]:
periods = list(df['period-denser'].unique())
most_common_lemmas_total = pd.DataFrame()
lemmas_length_total = 0

for period in periods:
    print(f'\n{period}\n')
    
    most_common_lemmas = pd.read_csv(f'../lemmatized/{period}/freq_2grams.csv',index_col=0)[['lemma', 'count_by_period']]

    lemmas_length_by_period = most_common_lemmas['count_by_period'].sum()
    lemmas_length_total += lemmas_length_by_period


    if most_common_lemmas_total.empty:
            most_common_lemmas_total = most_common_lemmas.rename(columns={'count_by_period':f'count_by_period_{period}'})
    else:
        most_common_lemmas_total = pd.merge(left=most_common_lemmas_total, right=most_common_lemmas.rename(columns={'count_by_period':f'count_by_period_{period}'}), on='lemma', how='outer')


    # count_df = df_period.iloc[:,[i for i in range(len(df_period.columns)) if df_period.columns[i].startswith('count_')]].fillna(0)
    # df_period['count_by_period'] = count_df.apply(sum,axis=1)
    # df_period = df_period.sort_values(by=['count_by_period'], ascending=False).reset_index()
    # df_period.to_csv(f'../lemmatized/{period}/freq_lemmas.csv')
    
    # most_common_lemmas_for_period["relative_count_by_period"] = most_common_lemmas_for_period["count_by_period"]/lemmas_length_by_period
most_common_lemmas_total.to_csv(f"../lemmatized/freq_2grams.csv")
    


николай ii


временное правительство


ленин - революция


ленин - поздние годы


сталин - довоенные годы


сталин - великая отечественная война


сталин - послевоенные годы


маленков


хрущев


брежнев


горбачев


ельцин 1991-1996


ельцин 1996-1999


путин 2000-2004


путин 2000-2008


медведев


путин 2012-2018


путин 2018-2024



In [68]:
df_total = pd.read_csv(f"../lemmatized/freq_2grams.csv", index_col=0)
count_df = df_total.iloc[:,[i for i in range(len(df_total.columns)) if df_total.columns[i].startswith('count_')]].fillna(0)
df_total['count_total'] = count_df.apply(sum,axis=1)
df_total = df_total.sort_values(by=['count_total'], ascending=False).reset_index().drop(columns=['index'])
df_total.to_csv(f"../lemmatized/freq_2grams.csv")


3grams

In [70]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    patterns = list(df[
        df['period-denser'] == period
    ]['pattern'].unique())
    most_common_lemmas_for_period = pd.DataFrame()
    
    lemmas_length_by_period = 0
    for pattern in patterns:
        print(f'\n{pattern}\n')
        filenames_for_pattern_period = df[
            (df['pattern'] == pattern)
        &
            (df['period-denser'] == period)
        ]['filename'].to_list()
        # lemmas = count_lemmas_total(filenames_for_pattern_period)

        lemmas_folder = f"../lemmatized/{period}/{pattern}"
        # Path(lemmas_folder).mkdir(parents=True, exist_ok=True)        
        lemmas = []
        with open(lemmas_folder+'/total_3grams.txt', 'r') as f:
            lemmas = (f.read()).split('\n')

        lemmas_length = len(lemmas)
        lemmas_length_by_period += lemmas_length
        most_common_lemmas = pd.DataFrame(
            [
                (
                    lemma_count[0],
                    lemma_count[1],
                    lemma_count[1]/lemmas_length*1_000_000
                    )
                for lemma_count in Counter(lemmas).most_common()
                ],columns=['lemma', 'count', 'relative_count']
            )
        if most_common_lemmas_for_period.empty:
            most_common_lemmas_for_period = most_common_lemmas.rename(columns={'count':f'count_{pattern}', 'relative_count': f'relative_count_{pattern}'})
        else:
            most_common_lemmas_for_period = pd.merge(left=most_common_lemmas_for_period, right=most_common_lemmas.rename(columns={'count':f'count_{pattern}', 'relative_count': f'relative_count_{pattern}'}), on='lemma', how='outer')

        most_common_lemmas.to_csv(lemmas_folder+'/freq_3grams.csv')
        
    
    # most_common_lemmas_for_period["relative_count_by_period"] = most_common_lemmas_for_period["count_by_period"]/lemmas_length_by_period
    most_common_lemmas_for_period.to_csv(f"../lemmatized/{period}/freq_3grams.csv")
    
    


николай ii


0111


0110


временное правительство


1011


1010


ленин - революция


1011


1010


1111


0110


1110


0111


0100


1000


ленин - поздние годы


1110


0100


1000


1001


1010


0111


0000


сталин - довоенные годы


1010


0000


0100


0111


1000


1011


сталин - великая отечественная война


0101


0110


0100


сталин - послевоенные годы


0110


0101


0100


1010


0010


маленков


0010


1010


хрущев


1010


0100


1000


брежнев


0100


1000


1010


1110


горбачев


0100


0000


1000


1010


0101


1110


ельцин 1991-1996


0101


1110


1010


0100


0110


ельцин 1996-1999


1010


0100


0101


путин 2000-2004


0100


0111


путин 2000-2008


0100


0111


медведев


0101


0100


0111


путин 2012-2018


0100


0101


0110


путин 2018-2024


0100


0101


0111


0010



In [71]:
periods = list(df['period-denser'].unique())
for period in periods:
    print(f'\n{period}\n')
    
    df_period = pd.read_csv(f'../lemmatized/{period}/freq_3grams.csv',index_col=0)

    count_df = df_period.iloc[:,[i for i in range(len(df_period.columns)) if df_period.columns[i].startswith('count_')]].fillna(0)
    df_period['count_by_period'] = count_df.apply(sum,axis=1)
    df_period = df_period.sort_values(by=['count_by_period'], ascending=False).reset_index()
    df_period.to_csv(f'../lemmatized/{period}/freq_3grams.csv')



николай ii


временное правительство


ленин - революция


ленин - поздние годы


сталин - довоенные годы


сталин - великая отечественная война


сталин - послевоенные годы


маленков


хрущев


брежнев


горбачев


ельцин 1991-1996


ельцин 1996-1999


путин 2000-2004


путин 2000-2008


медведев


путин 2012-2018


путин 2018-2024



In [72]:
periods = list(df['period-denser'].unique())
most_common_lemmas_total = pd.DataFrame()
lemmas_length_total = 0

for period in periods:
    print(f'\n{period}\n')
    
    most_common_lemmas = pd.read_csv(f'../lemmatized/{period}/freq_3grams.csv',index_col=0)[['lemma', 'count_by_period']]

    lemmas_length_by_period = most_common_lemmas['count_by_period'].sum()
    lemmas_length_total += lemmas_length_by_period


    if most_common_lemmas_total.empty:
            most_common_lemmas_total = most_common_lemmas.rename(columns={'count_by_period':f'count_by_period_{period}'})
    else:
        most_common_lemmas_total = pd.merge(left=most_common_lemmas_total, right=most_common_lemmas.rename(columns={'count_by_period':f'count_by_period_{period}'}), on='lemma', how='outer')


    # count_df = df_period.iloc[:,[i for i in range(len(df_period.columns)) if df_period.columns[i].startswith('count_')]].fillna(0)
    # df_period['count_by_period'] = count_df.apply(sum,axis=1)
    # df_period = df_period.sort_values(by=['count_by_period'], ascending=False).reset_index()
    # df_period.to_csv(f'../lemmatized/{period}/freq_lemmas.csv')
    
    # most_common_lemmas_for_period["relative_count_by_period"] = most_common_lemmas_for_period["count_by_period"]/lemmas_length_by_period
most_common_lemmas_total.to_csv(f"../lemmatized/freq_3grams.csv")
    


николай ii


временное правительство


ленин - революция


ленин - поздние годы


сталин - довоенные годы


сталин - великая отечественная война


сталин - послевоенные годы


маленков


хрущев


брежнев


горбачев


ельцин 1991-1996


ельцин 1996-1999


путин 2000-2004


путин 2000-2008


медведев


путин 2012-2018


путин 2018-2024



In [73]:
df_total = pd.read_csv(f"../lemmatized/freq_3grams.csv", index_col=0)
count_df = df_total.iloc[:,[i for i in range(len(df_total.columns)) if df_total.columns[i].startswith('count_')]].fillna(0)
df_total['count_total'] = count_df.apply(sum,axis=1)
df_total = df_total.sort_values(by=['count_total'], ascending=False).reset_index().drop(columns=['index'])
df_total.to_csv(f"../lemmatized/freq_3grams.csv")


In [33]:
dfg = pd.read_csv('../lemmatized/горбачев/freq_lemmas.csv',index_col=0)
dfg

,lemma,count_0100,relative_count_0100,count_0000,relative_count_0000,count_1000,relative_count_1000,count_1010,relative_count_1010,count_0101,relative_count_0101,count_1110,relative_count_1110
0,",",1.0,18.484972,1.0,105.898549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-,500.0,9242.485859,181.0,19167.637403,3250.0,13765.528575,892.0,18893.101476,49.0,12904.924941,5.0,14164.305949
2,.вышестоящего,NaN,NaN,NaN,NaN,NaN,NaN,1.0,21.180607,NaN,NaN,NaN,NaN
3,.нашей,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN
4,.партии,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21107,ёмкий,NaN,NaN,NaN,NaN,3.0,12.706642,NaN,NaN,NaN,NaN,NaN,NaN
21108,ёмкость,NaN,NaN,NaN,NaN,2.0,8.471095,2.0,42.361214,NaN,NaN,NaN,NaN
21109,•парторганизации,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN
21110,••,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,526.731630,NaN,NaN


In [47]:
count_df = dfg.iloc[:,[i for i in range(len(dfg.columns)) if dfg.columns[i].startswith('count_')]].fillna(0)
dfg['count_by_period'] = count_df.apply(sum,axis=1)
dfg = dfg.sort_values(by=['count_by_period'], ascending=False).reset_index()
dfg


,index,lemma,count_0100,relative_count_0100,count_0000,relative_count_0000,count_1000,relative_count_1000,count_1010,relative_count_1010,count_0101,relative_count_0101,count_1110,relative_count_1110,count_by_period
0,1,-,500.0,9242.485859,181.0,19167.637403,3250.0,13765.528575,892.0,18893.101476,49.0,12904.924941,5.0,14164.305949,14631.0
1,11722,партия,416.0,7689.748235,51.0,5400.826009,2720.0,11520.688531,488.0,10336.136234,4.0,1053.463260,1.0,2832.861190,11040.0
2,18650,товарищ,177.0,3271.839994,12.0,1270.782590,2422.0,10258.495449,7.0,148.264249,NaN,NaN,NaN,NaN,7854.0
3,18077,страна,309.0,5711.856261,47.0,4977.231812,1412.0,5980.592722,297.0,6290.640290,24.0,6320.779563,2.0,5665.722380,6273.0
4,11720,партийный,230.0,4251.543495,10.0,1058.985492,1478.0,6260.138841,318.0,6735.433038,5.0,1316.829076,NaN,NaN,6123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21107,4671,забывала,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN,3.0
21108,4672,забывалась,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN,3.0
21109,4740,завозятся,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN,3.0
21110,4744,завуалированный,NaN,NaN,NaN,NaN,1.0,4.235547,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [41]:
dfg.columns[3]

'count_0000'

In [39]:
len(dfg)

21112

In [33]:
with open('../lemmatized/горбачев/0100/total.txt', 'r') as f:
    lemmas = (f.read()).split('\n')
    print(lemmas)
lemmas_cleaned = [morph.parse(lemma)[0].normal_form for lemma in lemmas]
print(lemmas_cleaned)

['политический', 'доклад', 'центральный', 'комитет', 'съезду', 'коммунистический', 'партия', 'советский', 'союз', 'доклад', 'генеральный', 'секретарь', 'товарищ', 'товарищ', 'делегат', 'уважаемые', 'гость', 'съезд', 'кпсс', 'собраться', 'крутой', 'перелом', 'жизнь', 'страна', 'современный', 'мир', 'целое', 'начинать', 'работа', 'чувство', 'глубокий', 'понимание', 'ответственность', 'партия', 'советский', 'народ', 'задача', 'осмыслить', 'переживаемое', 'время', 'выработать', 'реалистический', 'взвешенную', 'программа', 'действие', 'соединить', 'величие', 'цель', 'реализм', 'возможность', 'план', 'партия', 'надеждами', 'чаяниями', 'человек', 'решение', 'xxvii', 'съезд', 'определить', 'характер', 'темп', 'движение', 'год', 'десятилетие', 'движение', 'новый', 'состояние', 'советский', 'социалистический', 'общество', 'съезд', 'предстоять', 'обсудить', 'принять', 'новый', 'редакция', 'программа', 'изменение', 'устав', 'партия', 'основной', 'направление', 'развитие', 'народный', 'хозяйство', 

In [ ]:
df[
    df['pattern'].isin(['0100'])
].groupby(by=['period-denser'])['token_count'].sum().sort_values(ascending=False)

In [13]:
df[
    df['pattern'].isin(['0100'])
]['filename'].to_list()

['1918.03.07_ленин_выступление.отчет.доклад_политический отчет цк_съезд ркпб 7',
 '1918.03.08_ленин_выступление.фрагмент_вопрос о брестском мире_съезд ркпб 7.брестский мир.1 мировая война',
 '1919.03.18_ленин_выступление.отчет.фрагмент_отчет цк_съезд ркпб 8',
 '1919.03.18_ленин_выступление.фрагмент_о деятельности федерации иностранных групп_съезд ркпб 8',
 '1920.09.22_ленин_выступление.отчет_политический отчет цк_съезд ркпб 9',
 '1921.03.05_ленин_выступление.отчет.фрагмент_политический отчет_съезд ркпб 10',
 '1922.03.27_ленин_выступление.фрагмент.отчет_политический отчет_съезд ркпб 11',
 '1923.04.17_сталин_выступление.отчет_организационный отчет_съезд ркпб 12',
 '1927.12.03_сталин_выступление.отчет_политический отчет цк_съезд вкпб 15',
 '1930.06.27_сталин_выступление.отчет_политический отчет цк_съезд вкпб 16',
 '1934.01.26_сталин_выступление.отчет.доклад_отчетный доклад о работе цк_съезд вкпб 17',
 '1939.03.10_сталин_выступление.отчет.доклад_отчетный доклад цк_съезд вкпб 18',
 '1941.11

In [14]:
df[
    df['pattern'].isin(['0100'])
].groupby(by=['period-denser']).count().sort_values(by=['filename'], ascending=False)['filename']

period-denser
горбачев                                10
путин 2018-2024                         10
ельцин 1991-1996                         9
путин 2012-2018                          8
брежнев                                  7
ленин - поздние годы                     6
медведев                                 6
путин 2000-2008                          6
путин 2000-2004                          5
ельцин 1996-1999                         4
сталин - довоенные годы                  4
сталин - послевоенные годы               4
ленин - революция                        2
сталин - великая отечественная война     2
хрущев                                   2
Name: filename, dtype: int64

In [15]:
df[
    df['pattern'].isin(['0100'])
].groupby(by=['period-denser'])['token_count'].sum().sort_values(ascending=False)

period-denser
брежнев                                 157481
горбачев                                109659
сталин - довоенные годы                  99838
сталин - послевоенные годы               85243
путин 2018-2024                          77360
ельцин 1991-1996                         75672
хрущев                                   70347
путин 2012-2018                          67739
ельцин 1996-1999                         59651
медведев                                 57277
путин 2000-2008                          32131
путин 2000-2004                          29451
ленин - поздние годы                     21955
ленин - революция                         9065
сталин - великая отечественная война      6266
Name: token_count, dtype: int64

In [20]:
df.groupby(by=['pattern']).count().sort_values(by=['filename'], ascending=False)['filename']

pattern
1010    113
0100     82
0111     36
0101     30
0110     29
1110     29
1000     19
1011     10
1111      7
0000      6
0010      1
Name: filename, dtype: int64

In [19]:
df.groupby(by=['pattern'])['token_count'].sum().sort_values(ascending=False)

pattern
1000    1010137
0100     871106
1010     338154
0101      55708
0111      50755
0110      22511
1110      20467
0000       6888
1011       5808
1111       5277
0010        788
Name: token_count, dtype: int64

In [25]:
df.groupby(by=['period-denser'])['token_count'].sum().sort_values(ascending=False)

period-denser
горбачев                                708348
брежнев                                 583204
сталин - довоенные годы                 178968
хрущев                                  173138
путин 2018-2024                         135208
ельцин 1991-1996                        103461
сталин - послевоенные годы               92392
путин 2012-2018                          75212
медведев                                 67869
ельцин 1996-1999                         60828
ленин - поздние годы                     53849
ленин - революция                        51685
путин 2000-2008                          43274
путин 2000-2004                          34964
сталин - великая отечественная война     12277
николай ii                                6972
временное правительство                   4570
маленков                                  1380
Name: token_count, dtype: int64

In [18]:
df[
    df['pattern'].isin(['0100'])
][
    df['period-denser'].isin(['горбачев'])
]['filename'].to_list()

/var/folders/bs/37p3rgss433755h614xs_rym0000gn/T/ipykernel_15683/2838310023.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[


['1986.02.25_горбачев_выступление.отчет.доклад_политический доклад_съезд кпсс 27',
 '1986.03.06_горбачев_выступление_заключительная речь_съезд кпсс 27',
 '1988.06.28_горбачев_выступление.доклад_задачи по углублению перестройки_партконференция19',
 '1988.07.01_горбачев_выступление_заключительная речь_партконференция19',
 '1988.12.07_горбачев_выступление_выступление в оон',
 '1989.04.27_ельцин_выступление_к 1 съезду народных депутатов_съезд народных депутатов 1',
 '1989.07.06_горбачев_выступление_выступление в совете европы',
 '1990.07.02_горбачев_выступление.доклад_политический отчет_съезд кпсс 28',
 '1990.11.13_ельцин_выступление.стенограмма.ocr_на 2 сессии верховного совета рсфср_сессия верховного совета рсфср',
 '1991.05.21_ельцин_выступление_на 4 съезде народных депутатов_съезд народных депутатов 4']

In [30]:
df[df['period-denser'].isin(['горбачев'])]

,filename,date,authors,genres,names,topics,multiple_authors,leader_in_authors,is_written_text,addressed_to_people,...,has_major_topic,pattern,year,is_fragment,distorted,full_pattern,token_count,filename_raw,period,period-denser
228,1986.02.25_горбачев кпсс27_выступление.отчет.д...,1986.02.25,[горбачев кпсс27],"[выступление, отчет, доклад]",[политический доклад],[политический доклад],True,False,False,False,...,False,1000,1986,False,False,10000000,38936,1986.02.25_горбачев-кпсс27_выступление.отчет.д...,горбачев,горбачев
229,1986.03.03_рыжков кпсс27_выступление.отчет.док...,1986.03.03,[рыжков кпсс27],"[выступление, отчет, доклад]",[эконом соц развитие],[эконом соц развитие],True,False,False,False,...,False,1000,1986,False,False,10000000,16804,1986.03.03_рыжков-кпсс27_выступление.отчет.док...,горбачев,горбачев
230,1986.03.06_горбачев кпсс27_выступление_заключи...,1986.03.06,[горбачев кпсс27],[выступление],[заключительная речь],[заключительная речь],True,False,False,False,...,False,1000,1986,False,False,10000000,2178,1986.03.06_горбачев-кпсс27_выступление_заключи...,горбачев,горбачев
231,1986.03.06_кпсс27_выступление_обсуждение докла...,1986.03.06,[кпсс27],[выступление],[обсуждение доклада эконом развития],[обсуждение доклада эконом развития],True,False,False,False,...,False,1000,1986,False,False,10000000,66163,1986.03.06_кпсс27_выступление_обсуждение-докла...,горбачев,горбачев
232,1986.03.06_кпсс27_выступление_обсуждение полит...,1986.03.06,[кпсс27],[выступление],[обсуждение политического доклада],[обсуждение политического доклада],True,False,False,False,...,False,1000,1986,False,False,10000000,155775,1986.03.06_кпсс27_выступление_обсуждение-полит...,горбачев,горбачев
233,1986.03.07_кпсс27_постановление_по докладу соц...,1986.03.07,[кпсс27],[постановление],[по докладу соц эконом развития],[по докладу соц эконом развития],True,False,True,False,...,False,1010,1986,False,False,10100000,25565,1986.03.07_кпсс27_постановление_по-докладу-соц...,горбачев,горбачев
234,1986.03.07_кпсс27_программа_программа кпсс,1986.03.07,[кпсс27],[программа],[программа кпсс],[программа кпсс],True,False,True,False,...,False,1010,1986,False,False,10100000,24089,1986.03.07_кпсс27_программа_программа-кпсс.txt,горбачев,горбачев
235,1986.03.07_кпсс27_резолюция_по политическому д...,1986.03.07,[кпсс27],[резолюция],[по политическому докладу],[по политическому докладу],True,False,True,False,...,False,1010,1986,False,False,10100000,8500,1986.03.07_кпсс27_резолюция_по-политическому-д...,горбачев,горбачев
236,1986.03.07_кпсс27_устав_устав кпсс,1986.03.07,[кпсс27],[устав],[устав кпсс],[устав кпсс],True,False,True,False,...,False,1010,1986,False,False,10100000,6924,1986.03.07_кпсс27_устав_устав-кпсс.txt,горбачев,горбачев
237,1988.06.28_горбачев партконференция19_выступле...,1988.06.28,[горбачев партконференция19],"[выступление, доклад]",[задачи по углублению перестройки],[задачи по углублению перестройки],True,False,False,False,...,False,1000,1988,False,False,10000000,29009,1988.06.28_горбачев-партконференция19_выступле...,горбачев,горбачев


In [ ]:
df[
    df['pattern'].isin(['0100'])
]['filename'].to_list()

In [ ]:
dirpath = 'magazines_pre_add'
for filename in os.listdir(dirpath):
  filepath = os.path.join(dirpath, filename)
  if os.path.isfile(filepath):
    try:
      with open(filepath, encoding = 'utf-8') as fd:
        text = fd.read()
      text = text.lower()
      textforspacy = nlp(text)
      for token in textforspacy:
        if token.lemma_ not in stopwordsspacy:
          if token.tag_ == 'NOUN':
            noun_magazines.append(token.lemma_)
          elif token.tag_ == 'ADJ':
            adj_magazines.append(token.lemma_)
          elif token.tag_ == 'VERB':
            verb_magazines.append(token.lemma_)
      print(filename)                           
    except:
      print(f'С файлом {filename} проблемка')    